# Collaborative filtering on Google Analytics data

This notebook demonstrates how to implement a WALS matrix refactorization approach to do collaborative filtering.

In [1]:
import os
output = os.popen("gcloud config get-value project").readlines()
project_name = output[0][:-1]

# change these to try this notebook out
PROJECT = project_name
BUCKET = project_name
#BUCKET = BUCKET.replace("qwiklabs-gcp-", "inna-bckt-")
REGION = 'europe-west1'  ## note: Cloud ML Engine not availabe in europe-west3!

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

print(PROJECT)
print(BUCKET)
print(REGION)

qwiklabs-gcp-3dac8d620decbcdb
qwiklabs-gcp-3dac8d620decbcdb
europe-west1


In [2]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [3]:
import tensorflow as tf
print(tf.__version__)

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


## Create raw dataset
<p>
For collaborative filtering, we don't need to know anything about either the users or the content. Essentially, all we need to know is userId, itemId, and rating that the particular user gave the particular item.
<p>
In this case, we are working with newspaper articles. The company doesn't ask their users to rate the articles. However, we can use the time-spent on the page as a proxy for rating.
<p>
Normally, we would also add a time filter to this ("latest 7 days"), but our dataset is itself limited to a few days.

In [4]:
import google.datalab.bigquery as bq

sql="""
#standardSQL
WITH visitor_page_content AS (

   SELECT  
     fullVisitorID,
     (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS latestContentId,  
     (LEAD(hits.time, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) - hits.time) AS session_duration 
   FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"

   GROUP BY   
     fullVisitorId, latestContentId, hits.time
     )

# aggregate web stats
SELECT   
  fullVisitorID as visitorId,
  latestContentId as contentId,
  SUM(session_duration) AS session_duration 
 
FROM visitor_page_content
  WHERE latestContentId IS NOT NULL 
  GROUP BY fullVisitorID, latestContentId
  HAVING session_duration > 0
  ORDER BY latestContentId 
"""

df = bq.Query(sql).execute().result().to_dataframe()
df.head()

,visitorId,contentId,session_duration
0,7337153711992174438,100074831,44652
1,5190801220865459604,100170790,1214205
2,5874973374932455844,100510126,32109
3,2293633612703952721,100510126,47744
4,1173698801255170595,100676857,10512


In [7]:
stats = df.describe()
stats.round(2)

,session_duration
count,278913.00
mean,127218.80
std,234643.86
min,1.00
25%,17095.00
50%,57938.00
75%,129393.00
max,7690598.00


In [9]:
# the rating is the session_duration scaled to be in the range 0-1.  This will help with training.
df['rating'] = 0.3 * (1 + (df['session_duration'] - stats.loc['50%', 'session_duration'])/stats.loc['50%', 'session_duration'])
df.loc[df['rating'] > 1, 'rating'] = 1
df.describe().round(3)

,session_duration,rating
count,278913.000,278913.000
mean,127218.799,0.402
std,234643.856,0.350
min,1.000,0.000
25%,17095.000,0.089
50%,57938.000,0.300
75%,129393.000,0.670
max,7690598.000,1.000


In [10]:
del df['session_duration']

In [11]:
%%bash
rm -rf data
mkdir data

In [12]:
df.to_csv('data/collab_raw.csv', index=False, header=False)

In [13]:
!head data/collab_raw.csv

7337153711992174438,100074831,0.23120577168697573
5190801220865459604,100170790,1.0
5874973374932455844,100510126,0.1662587593634575
2293633612703952721,100510126,0.24721598950602366
1173698801255170595,100676857,0.05443059822568952
7262596434087807556,100735153,0.2313403983568642
883397426232997550,10083328,0.9450274431288619
1808867070685560283,100906145,1.0
7615995624631762562,100906145,0.48231040077324033
5519169380728479914,100915139,0.199485657081708


## Create dataset for WALS
<p>
The raw dataset (above) won't work for WALS:
<ol>
<li> The userId and itemId have to be 0,1,2 ... so we need to create a mapping from visitorId (in the raw data) to userId and contentId (in the raw data) to itemId.
<li> We will need to save the above mapping to a file because at prediction time, we'll need to know how to map the contentId in the table above to the itemId.
<li> We'll need two files: a "rows" dataset where all the items for a particular user are listed; and a "columns" dataset where all the users for a particular item are listed.
</ol>

<p>

### Mapping

In [14]:
import pandas as pd
import numpy as np
def create_mapping(values, filename):
  with open(filename, 'w') as ofp:
    value_to_id = {value:idx for idx, value in enumerate(values.unique())}
    for value, idx in value_to_id.items():
      ofp.write('{},{}\n'.format(value, idx))
  return value_to_id

df = pd.read_csv('data/collab_raw.csv',
                 header=None,
                 names=['visitorId', 'contentId', 'rating'],
                dtype={'visitorId': str, 'contentId': str, 'rating': np.float})
df.to_csv('data/collab_raw.csv', index=False, header=False)
user_mapping = create_mapping(df['visitorId'], 'data/users.csv')
item_mapping = create_mapping(df['contentId'], 'data/items.csv')

In [15]:
!head -3 data/*.csv

==> data/collab_raw.csv <==
7337153711992174438,100074831,0.2312057716869757
5190801220865459604,100170790,1.0
5874973374932455844,100510126,0.1662587593634575

==> data/items.csv <==
236941062,1402
299786600,4672
243874813,1540

==> data/users.csv <==
1672553935406836733,2849
3588200922529727879,41146
7789950935531938708,60125


In [16]:
df['userId'] = df['visitorId'].map(user_mapping.get)
df['itemId'] = df['contentId'].map(item_mapping.get)

In [17]:
mapped_df = df[['userId', 'itemId', 'rating']]
mapped_df.to_csv('data/collab_mapped.csv', index=False, header=False)
mapped_df.head()

,userId,itemId,rating
0,0,0,0.231206
1,1,1,1.000000
2,2,2,0.166259
3,3,2,0.247216
4,4,3,0.054431


### Creating rows and columns datasets

In [18]:
import pandas as pd
import numpy as np
mapped_df = pd.read_csv('data/collab_mapped.csv', header=None, names=['userId', 'itemId', 'rating'])
mapped_df.head()

,userId,itemId,rating
0,0,0,0.231206
1,1,1,1.000000
2,2,2,0.166259
3,3,2,0.247216
4,4,3,0.054431


In [21]:
NITEMS = np.max(mapped_df['itemId']) + 1
NUSERS = np.max(mapped_df['userId']) + 1
mapped_df['rating'] = np.round(mapped_df['rating'].values, 2)
print('{} items, {} users, {} interactions'.format( NITEMS, NUSERS, len(mapped_df) ))
mapped_df.head()

5668 items, 82802 users, 278913 interactions


,userId,itemId,rating
0,0,0,0.23
1,1,1,1.00
2,2,2,0.17
3,3,2,0.25
4,4,3,0.05


In [22]:
grouped_by_items = mapped_df.groupby('itemId')
iter = 0
for item, grouped in grouped_by_items:
  print(item, grouped['userId'].values, grouped['rating'].values)
  iter = iter + 1
  if iter > 5:
    break

(0, array([0]), array([0.23]))
(1, array([1]), array([1.]))
(2, array([2, 3]), array([0.17, 0.25]))
(3, array([4]), array([0.05]))
(4, array([5]), array([0.23]))
(5, array([6]), array([0.95]))


In [23]:
import tensorflow as tf
grouped_by_items = mapped_df.groupby('itemId')
with tf.python_io.TFRecordWriter('data/users_for_item') as ofp:
  for item, grouped in grouped_by_items:
    example = tf.train.Example(features=tf.train.Features(feature={
          'key': tf.train.Feature(int64_list=tf.train.Int64List(value=[item])),
          'indices': tf.train.Feature(int64_list=tf.train.Int64List(value=grouped['userId'].values)),
          'values': tf.train.Feature(float_list=tf.train.FloatList(value=grouped['rating'].values))
        }))
    ofp.write(example.SerializeToString())      

In [24]:
grouped_by_users = mapped_df.groupby('userId')
with tf.python_io.TFRecordWriter('data/items_for_user') as ofp:
  for user, grouped in grouped_by_users:
    example = tf.train.Example(features=tf.train.Features(feature={
          'key': tf.train.Feature(int64_list=tf.train.Int64List(value=[user])),
          'indices': tf.train.Feature(int64_list=tf.train.Int64List(value=grouped['itemId'].values)),
          'values': tf.train.Feature(float_list=tf.train.FloatList(value=grouped['rating'].values))
        }))
    ofp.write(example.SerializeToString())      

In [25]:
grouped_by_users = mapped_df.groupby('userId')
N = 0
with tf.python_io.TFRecordWriter('data/items_for_user_subset') as ofp:
  for user, grouped in grouped_by_users:
    example = tf.train.Example(features=tf.train.Features(feature={
          'key': tf.train.Feature(int64_list=tf.train.Int64List(value=[user])),
          'indices': tf.train.Feature(int64_list=tf.train.Int64List(value=grouped['itemId'].values)),
          'values': tf.train.Feature(float_list=tf.train.FloatList(value=grouped['rating'].values))
        }))
    ofp.write(example.SerializeToString())    
    N = N + 1
    if N > 20:
      break

In [26]:
!ls -lrt data

total 31836
-rw-r--r-- 1 root root 13121887 Mar 20 12:45 collab_raw.csv
-rw-r--r-- 1 root root  2131923 Mar 20 12:45 users.csv
-rw-r--r-- 1 root root    82193 Mar 20 12:45 items.csv
-rw-r--r-- 1 root root  7794658 Mar 20 12:48 collab_mapped.csv
-rw-r--r-- 1 root root  2244880 Mar 20 12:51 users_for_item
-rw-r--r-- 1 root root  7207025 Mar 20 12:52 items_for_user
-rw-r--r-- 1 root root     1747 Mar 20 12:52 items_for_user_subset


To summarize, we created the following data files from collab_raw.csv:

* `collab_mapped.csv` is essentially the same data as in `collab_raw.csv` except that `visitorId` and `contentId` which are business-specific have been mapped to `userId` and `itemId` which are enumerated in 0,1,2,....  The mappings themselves are stored in `items.csv` and `users.csv` so that they can be used during inference.
* `users_for_item` contains all the users/ratings for each item in TFExample format
* `items_for_user` contains all the items/ratings for each user in TFExample format


## Train with WALS

Once you have the dataset, do matrix factorization with WALS using the [WALSMatrixFactorization](https://www.tensorflow.org/versions/master/api_docs/python/tf/contrib/factorization/WALSMatrixFactorization) in the contrib directory.
This is an estimator model, so it should be relatively familiar.
<p>
As usual, we write an input_fn to provide the data to the model, and then create the Estimator to do train_and_evaluate.
Because it is in contrib and hasn't moved over to tf.estimator yet, we use tf.contrib.learn.Experiment to handle the training loop.

In [27]:
import os
import tensorflow as tf
from tensorflow.python.lib.io import file_io
from tensorflow.contrib.factorization import WALSMatrixFactorization
  
def read_dataset(mode, args):
  def decode_example(protos, vocab_size):
    #TODO
    features = {'key': tf.FixedLenFeature([1], tf.int64),   ## one user (or one item)
                'indices': tf.VarLenFeature(dtype = tf.int64),  ## multiple items (or users) that the user (item) has interacted with
                'values': tf.VarLenFeature(dtype = tf.float32)  ## ratings of these items (or users) in the above interactions
               }
    ## read tf-record file examples in the given format above:
    parsed_features = tf.parse_single_example(protos, features)
    ## convert into sparse tensor as needed for WALS:
    values = tf.sparse_merge(
      parsed_features['indices'], 
      parsed_features['values'],
      vocab_size = vocab_size
    )
    # Save key to remap after batching
    # This is a temporary workaround to assign correct row numbers in each batch.
    # You can ignore details of this part and remap_keys().
    key = parsed_features['key']
    decoded_sparse_tensor = tf.SparseTensor(
      indices = tf.concat([values.indices, [key]], axis = 0),
      values = tf.concat([values.values, [0.0]], axis = 0),
      dense_shape = values.dense_shape
    )
    return decoded_sparse_tensor
  
  
  def remap_keys(sparse_tensor):
    # Current indices of our SparseTensor that we need to fix
    bad_indices = sparse_tensor.indices
    # Current values of our SparseTensor that we need to fix
    bad_values = sparse_tensor.values 
  
    # Group by the batch_indices and get the count for each  
    size = tf.segment_sum(data = tf.ones_like(bad_indices[:,0], dtype = tf.int64), segment_ids = bad_indices[:,0]) - 1
    # The number of batch_indices (this should be batch_size unless it is a partially full batch)
    length = tf.shape(size, out_type = tf.int64)[0]
    # Finds the cumulative sum which we can use for indexing later
    cum = tf.cumsum(size)
    # The offsets between each example in the batch due to our concatentation of the keys in the decode_example method
    length_range = tf.range(start = 0, limit = length, delta = 1, dtype = tf.int64)
    # Indices of the SparseTensor's indices member of the rows we added by the concatentation of our keys in the decode_example method
    cum_range = cum + length_range

    # The keys that we have extracted back out of our concatentated SparseTensor
    gathered_indices = tf.squeeze(tf.gather(bad_indices, cum_range)[:,1])

    # The enumerated row indices of the SparseTensor's indices member
    sparse_indices_range = tf.range(tf.shape(bad_indices, out_type = tf.int64)[0], dtype = tf.int64)

    # We want to find here the row indices of the SparseTensor's indices member that are of our actual data and not the concatentated rows
    # So we want to find the intersection of the two sets and then take the opposite of that
    x = sparse_indices_range
    s = cum_range

    # Number of multiples we are going to tile x, which is our sparse_indices_range
    tile_multiples = tf.concat([tf.ones(tf.shape(tf.shape(x)), dtype=tf.int64), tf.shape(s, out_type = tf.int64)], axis = 0)
    # Expands x, our sparse_indices_range, into a rank 2 tensor and then multiplies the rows by 1 (no copying) and the columns by the number of examples in the batch
    x_tile = tf.tile(tf.expand_dims(x, -1), tile_multiples)
    # Essentially a vectorized logical or, that we then negate
    x_not_in_s = ~tf.reduce_any(tf.equal(x_tile, s), -1)

    # The SparseTensor's indices that are our actual data by using the boolean_mask we just made above applied to the entire indices member of our SparseTensor
    selected_indices = tf.boolean_mask(tensor = bad_indices, mask = x_not_in_s, axis = 0)
    # Apply the same boolean_mask to the entire values member of our SparseTensor to get the actual values data
    selected_values = tf.boolean_mask(tensor = bad_values, mask = x_not_in_s, axis = 0)

    # Need to replace the first column of our selected_indices with keys, so we first need to tile our gathered_indices
    tiling = tf.tile(input = tf.expand_dims(gathered_indices[0], -1), multiples = tf.expand_dims(size[0] , -1))
    
    # We have to repeatedly apply the tiling to each example in the batch
    # Since it is jagged we cannot use tf.map_fn due to the stacking of the TensorArray, so we have to create our own custom version
    def loop_body(i, tensor_grow):
      return i + 1, tf.concat(values = [tensor_grow, tf.tile(input = tf.expand_dims(gathered_indices[i], -1), multiples = tf.expand_dims(size[i] , -1))], axis = 0)

    _, result = tf.while_loop(lambda i, tensor_grow: i < length, loop_body, [tf.constant(1, dtype = tf.int64), tiling])
    
    # Concatenate tiled keys with the 2nd column of selected_indices
    selected_indices_fixed = tf.concat([tf.expand_dims(result, -1), tf.expand_dims(selected_indices[:, 1], -1)], axis = 1)
    
    # Combine everything together back into a SparseTensor
    remapped_sparse_tensor = tf.SparseTensor(indices = selected_indices_fixed, values = selected_values, dense_shape = sparse_tensor.dense_shape)
    return remapped_sparse_tensor

    
  def parse_tfrecords(filename, vocab_size):
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
    else:
        num_epochs = 1 # end-of-input after this
    
    files = tf.gfile.Glob(os.path.join(args['input_path'], filename))
    
    # Create dataset from file list
    dataset = tf.data.TFRecordDataset(files)
    dataset = dataset.map(lambda x: decode_example(x, vocab_size))
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(args['batch_size'])
    dataset = dataset.map(lambda x: remap_keys(x))
    return dataset.make_one_shot_iterator().get_next()
  
  def _input_fn():
    features = {
      WALSMatrixFactorization.INPUT_ROWS: parse_tfrecords('items_for_user', args['nitems']),
      WALSMatrixFactorization.INPUT_COLS: parse_tfrecords('users_for_item', args['nusers']),
      WALSMatrixFactorization.PROJECT_ROW: tf.constant(True)
    }
    return features, None
  
  # just for developing line by line. You don't need this in production
  def _input_fn_subset():
    features = {
      WALSMatrixFactorization.INPUT_ROWS: parse_tfrecords('items_for_user_subset', args['nitems']),
      WALSMatrixFactorization.PROJECT_ROW: tf.constant(True)
    }
    return features, None
  
  def input_cols():
    return parse_tfrecords('users_for_item', args['nusers'])
  
  return _input_fn#_subset

This code is helpful in developing the input function. You don't need it in production.

In [28]:
def try_out():
  with tf.Session() as sess:
    fn = read_dataset(tf.estimator.ModeKeys.EVAL, 
                    {'input_path': 'data', 'batch_size': 8, 'nitems': 5668, 'nusers': 82802})
    feats, _ = fn()
    print(feats['input_rows'].eval())

try_out()

SparseTensorValue(indices=array([[   0,    0],
       [   0, 3485],
       [   0, 3542],
       [   1,    1],
       [   1, 2332],
       [   1, 3105],
       [   1, 4819],
       [   1, 4856],
       [   1, 4861],
       [   1, 5614],
       [   2,    2],
       [   2, 1441],
       [   3,    2],
       [   4,    3],
       [   5,    4],
       [   5, 1763],
       [   5, 4746],
       [   5, 4804],
       [   5, 4866],
       [   6,    5],
       [   6, 4994],
       [   6, 5473],
       [   6, 5501],
       [   7,    6]]), values=array([0.23, 0.05, 0.17, 1.  , 0.11, 0.55, 0.3 , 0.72, 0.46, 0.3 , 0.17,
       0.12, 0.25, 0.05, 0.23, 1.  , 0.41, 0.33, 0.21, 0.95, 0.62, 1.  ,
       0.16, 1.  ], dtype=float32), dense_shape=array([   8, 5668]))


In [29]:
def find_top_k(user, item_factors, k):
  all_items = tf.matmul(tf.expand_dims(user, 0), tf.transpose(item_factors))
  topk = tf.nn.top_k(all_items, k=k)
  return tf.cast(topk.indices, dtype=tf.int64)
    
def batch_predict(args):
  import numpy as np
  with tf.Session() as sess:
    #TODO
    estimator = tf.contrib.factorization.WALSMatrixFactorization(
      num_rows = args['nusers'],
      num_cols = args['nitems'],
      embedding_dimension = args['n_embeds'],
      model_dir = args['output_dir']
    )
    # this is how you would get the row factors for out-of-vocab user data
    #row_factors = list(estimator.get_projections(input_fn=read_dataset(tf.estimator.ModeKeys.EVAL, args)))
    #user_factors = tf.convert_to_tensor(np.array(row_factors))
    
    # but for in-vocab data, the row factors are already in the checkpoint
    user_factors = tf.convert_to_tensor(estimator.get_row_factors()[0]) # (nusers, nembeds)
    # in either case, we have to assume catalog doesn't change, so col_factors are read in
    item_factors = tf.convert_to_tensor(estimator.get_col_factors()[0])# (nitems, nembeds)
    
    # for each user, find the top K items
    topk = tf.squeeze(tf.map_fn(lambda user: find_top_k(user, item_factors, args['topk']), user_factors, dtype=tf.int64))
    with file_io.FileIO(os.path.join(args['output_dir'], 'batch_pred.txt'), mode='w') as f:
      for best_items_for_user in topk.eval():
        f.write(','.join(str(x) for x in best_items_for_user) + '\n')

# online prediction returns row and column factors as needed
def create_serving_input_fn(args):
  def for_user_embeddings(userId):
      # all items for this user (for user_embeddings)
      items = tf.range(args['nitems'], dtype=tf.int64)
      users = userId * tf.ones([args['nitems']], dtype=tf.int64)
      ratings = 0.1 * tf.ones_like(users, dtype=tf.float32)
      return items, users, ratings, tf.constant(True)
    
  def for_item_embeddings(itemId):
      # all users for this item (for item_embeddings)
      users = tf.range(args['nusers'], dtype=tf.int64)
      items = itemId * tf.ones([args['nusers']], dtype=tf.int64)
      ratings = 0.1 * tf.ones_like(users, dtype=tf.float32)
      return items, users, ratings, tf.constant(False)
    
  def serving_input_fn():
    feature_ph = {
        'userId': tf.placeholder(tf.int64, 1),
        'itemId': tf.placeholder(tf.int64, 1)
    }

    (items, users, ratings, project_row) = \
                  tf.cond(feature_ph['userId'][0] < tf.constant(0, dtype=tf.int64),
                          lambda: for_item_embeddings(feature_ph['itemId']),
                          lambda: for_user_embeddings(feature_ph['userId']))
    rows = tf.stack( [users, items], axis=1 )
    cols = tf.stack( [items, users], axis=1 )
    input_rows = tf.SparseTensor(rows, ratings, (args['nusers'], args['nitems']))
    input_cols = tf.SparseTensor(cols, ratings, (args['nusers'], args['nitems']))
    
    features = {
      WALSMatrixFactorization.INPUT_ROWS: input_rows,
      WALSMatrixFactorization.INPUT_COLS: input_cols,
      WALSMatrixFactorization.PROJECT_ROW: project_row
    }
    return tf.contrib.learn.InputFnOps(features, None, feature_ph)
  return serving_input_fn
        
def train_and_evaluate(args):
    train_steps = int(0.5 + (1.0 * args['num_epochs'] * args['nusers']) / args['batch_size'])
    steps_in_epoch = int(0.5 + args['nusers'] / args['batch_size'])
    print('Will train for {} steps, evaluating once every {} steps'.format(train_steps, steps_in_epoch))
    def experiment_fn(output_dir):
        return tf.contrib.learn.Experiment(
            tf.contrib.factorization.WALSMatrixFactorization(
                         num_rows=args['nusers'], num_cols=args['nitems'],
                         embedding_dimension=args['n_embeds'],
                         model_dir=args['output_dir']),
            train_input_fn=read_dataset(tf.estimator.ModeKeys.TRAIN, args),
            eval_input_fn=read_dataset(tf.estimator.ModeKeys.EVAL, args),
            train_steps=train_steps,
            eval_steps=1,
            min_eval_frequency=steps_in_epoch,
            export_strategies=tf.contrib.learn.utils.saved_model_export_utils.make_export_strategy(serving_input_fn=create_serving_input_fn(args))
        )

    from tensorflow.contrib.learn.python.learn import learn_runner
    learn_runner.run(experiment_fn, args['output_dir'])
    
    batch_predict(args)

In [30]:
import shutil
shutil.rmtree('wals_trained', ignore_errors=True)
train_and_evaluate({
    'output_dir': 'wals_trained',
    'input_path': 'data/',
    'num_epochs': 0.05,
    'nitems': 5668,
    'nusers': 82802,

    'batch_size': 512,
    'n_embeds': 10,
    'topk': 3
  })

Will train for 8 steps, evaluating once every 161 steps
Instructions for updating:
Use tf.estimator.train_and_evaluate.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6dd00d0a50>, '_model_dir': 'wals_trained', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction

Instructions for updating:
Please implement your own file name management.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6dba04df90>, '_model_dir': 'wals_trained', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}


In [31]:
!ls wals_trained

batch_pred.txt				     model.ckpt-1.data-00000-of-00001
checkpoint				     model.ckpt-1.index
eval					     model.ckpt-1.meta
events.out.tfevents.1553087338.ef45665e5d36  model.ckpt-8.data-00000-of-00001
export					     model.ckpt-8.index
graph.pbtxt				     model.ckpt-8.meta


In [32]:
!head wals_trained/batch_pred.txt

4509,4166,4399
954,4639,612
2484,2,4422
2,2484,2089
4630,432,2693
3657,292,3095
5464,5,1324
6,2484,1642
6,2484,1642
3135,4644,4780


## Run as a Python module

Let's run it as Python module for just a few steps.

In [33]:
%%bash
rm -rf wals.tar.gz wals_trained
gcloud ml-engine local train \
   --module-name=walsmodel.task \
   --package-path=${PWD}/walsmodel \
   -- \
   --output_dir=${PWD}/wals_trained \
   --input_path=${PWD}/data \
   --num_epochs=0.01 --nitems=5668 --nusers=82802 \
   --job-dir=./tmp

Will train for 2 steps, evaluating once every 162 steps


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Instructions for updating:
Use tf.estimator.train_and_evaluate.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd0b6ff3290>, '_model_dir': '/content/datalab/gcp-ml-02-advanced-ml-with-tf-on-gcp/05-recommen

## Get row and column factors

Once you have a trained WALS model, you can get row and column factors (user and item embeddings) using the serving input function that we exported.  We'll look at how to use these in the section on building a recommendation system using deep neural networks.

In [34]:
%writefile data/input.json
{"userId": 4, "itemId": -1}

Writing data/input.json


In [35]:
%%bash
MODEL_DIR=$(ls wals_trained/export/Servo | tail -1)
gcloud ml-engine local predict --model-dir=wals_trained/export/Servo/$MODEL_DIR --json-instances=data/input.json

PROJECTION
[0.0015905073378235102, 0.001691297860816121, 0.0002097050892189145, 7.143408993215417e-07, -0.002179093426093459, -0.0005687473458237946, -0.0023673921823501587, 0.0019316795514896512, 0.00080652761971578, -0.0007764641777612269]


  from ._conv import register_converters as _register_converters
2019-03-20 13:10:43.386673: W tensorflow/core/grappler/optimizers/arithmetic_optimizer.cc:1441] Failed to build SimpleGraphView.
2019-03-20 13:10:43.389557: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:586] Iteration = 0, topological sort failed with message: Non-existent input ^ConstantFoldingCtrl/cond_1/cond_1/ones/Less/Switch_1 for node ConstantFolding/cond_1/cond_1/ones/packed_const_axis
2019-03-20 13:10:43.390754: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:586] Iteration = 1, topological sort failed with message: Non-existent input ^ConstantFoldingCtrl/cond_1/cond_1/ones/Less/Switch_1 for node ConstantFolding/cond_1/cond_1/ones/packed_const_axis



In [36]:
%writefile data/input.json
{"userId": -1, "itemId": 4}

Overwriting data/input.json


In [37]:
%%bash
MODEL_DIR=$(ls wals_trained/export/Servo | tail -1)
gcloud ml-engine local predict --model-dir=wals_trained/export/Servo/$MODEL_DIR --json-instances=data/input.json

PROJECTION
[-0.0005234909476712346, 6.263308023335412e-05, 7.853408169467002e-05, 0.0002994881069753319, 9.925249469233677e-05, -0.00028537726029753685, -2.735387624852592e-06, -0.0001421043707523495, -0.0004419634351506829, -0.0001273992529604584]


  from ._conv import register_converters as _register_converters
2019-03-20 13:11:00.499831: W tensorflow/core/grappler/optimizers/arithmetic_optimizer.cc:1441] Failed to build SimpleGraphView.
2019-03-20 13:11:00.502954: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:586] Iteration = 0, topological sort failed with message: Non-existent input ^ConstantFoldingCtrl/cond_1/cond_1/ones/Less/Switch_1 for node ConstantFolding/cond_1/cond_1/ones/packed_const_axis
2019-03-20 13:11:00.503967: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:586] Iteration = 1, topological sort failed with message: Non-existent input ^ConstantFoldingCtrl/cond_1/cond_1/ones/Less/Switch_1 for node ConstantFolding/cond_1/cond_1/ones/packed_const_axis



## Run on Cloud

In [38]:
%%bash
gsutil -m cp data/* gs://${BUCKET}/wals/data

Copying file://data/collab_mapped.csv [Content-Type=text/csv]...
Copying file://data/collab_raw.csv [Content-Type=text/csv]...
Copying file://data/input.json [Content-Type=application/json]...
Copying file://data/items.csv [Content-Type=text/csv]...
Copying file://data/items_for_user [Content-Type=application/octet-stream]...
Copying file://data/items_for_user_subset [Content-Type=application/octet-stream]...
Copying file://data/users.csv [Content-Type=text/csv]...
Copying file://data/users_for_item [Content-Type=application/octet-stream]...
- [8/8 files][ 31.1 MiB/ 31.1 MiB] 100% Done                                    
Operation completed over 8 objects/31.1 MiB.                                     


In [39]:
%%bash
OUTDIR=gs://${BUCKET}/wals/model_trained
JOBNAME=wals_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=walsmodel.task \
   --package-path=${PWD}/walsmodel \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC_GPU \
   --runtime-version=$TFVERSION \
   -- \
   --output_dir=$OUTDIR \
   --input_path=gs://${BUCKET}/wals/data \
   --num_epochs=10 --nitems=5668 --nusers=82802 

gs://qwiklabs-gcp-3dac8d620decbcdb/wals/model_trained europe-west1 wals_190320_131125
jobId: wals_190320_131125
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [wals_190320_131125] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe wals_190320_131125

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs wals_190320_131125


This took <b>10 minutes</b> for me.

<pre>
# Copyright 2018 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>